# System Tables
- https://docs.databricks.com/en/admin/system-tables/index.html
- https://docs.databricks.com/en/admin/system-tables/billing.html
- https://docs.databricks.com/en/admin/system-tables/compute.html

# Run
- This notebook is entirelly sql; rec'd running against serverless sql warehouse for best experience.

In [0]:
%sql
-- DBUs by cluster
SELECT
  usage_date,
  sku_name,
  usage_metadata.cluster_id,
  SUM(usage_quantity) as total_dbu
FROM 
  system.billing.usage
WHERE 
  usage_metadata.cluster_id IN ('1216-145612-m94k4ybw')
  and usage_date >= DATEADD(day, -30, current_date)
  -- AND usage_unit = 'DBUs'
GROUP BY ALL
ORDER BY 
  usage_date DESC;

# Ganglia sytle metrics, e.g. CPU, Memory, Disk, Network

In [0]:
%sql
-- simple example
SELECT 
  cluster_id,
  instance_id,
  start_time,
  end_time,
  ROUND(cpu_user_percent + cpu_system_percent + cpu_wait_percent, 2) as total_cpu_percent,
  ROUND(mem_used_percent, 2) as memory_used_percent,
  ROUND(network_received_bytes/1024/1024, 2) as network_received_mb,
  ROUND(network_sent_bytes/1024/1024, 2) as network_sent_mb
FROM system.compute.node_timeline
WHERE cluster_id = '1216-145612-m94k4ybw'
  AND start_time >= CURRENT_TIMESTAMP - INTERVAL 1 hour
ORDER BY start_time DESC;

In [0]:
-- simple example
SELECT
  cluster_id,
  driver,
  AVG(cpu_user_percent + cpu_system_percent) AS avg_cpu_utilization,
  MAX(cpu_user_percent + cpu_system_percent) AS peak_cpu_utilization,
  AVG(cpu_wait_percent) AS avg_cpu_wait,
  MAX(cpu_wait_percent) AS max_cpu_wait
FROM
  system.compute.node_timeline
WHERE
  start_time >= CURRENT_TIMESTAMP - INTERVAL 24 hour
GROUP BY
  cluster_id,
  driver
ORDER BY
  avg_cpu_utilization DESC;

In [0]:
WITH base_metrics AS (
  SELECT 
    nt.workspace_id,
    nt.cluster_id,
    nt.instance_id,
    nt.driver,
    nt.start_time,
    -- CPU Analysis with combined metrics
    (nt.cpu_user_percent + nt.cpu_system_percent + nt.cpu_wait_percent) as total_cpu_percent,
    nt.cpu_wait_percent,
    
    -- Memory Analysis
    nt.mem_used_percent,
    nt.mem_swap_percent,
    
    -- Network Analysis (converted to MB for readability)
    nt.network_received_bytes/1024/1024 as network_received_mb,
    nt.network_sent_bytes/1024/1024 as network_sent_mb,
    
    -- Disk Analysis - extract minimum free space across all mount points
    -- This uses element_at to safely access map values
    LEAST(
      element_at(nt.disk_free_bytes_per_mount_point, '/'),
      element_at(nt.disk_free_bytes_per_mount_point, '/local_disk0'),
      element_at(nt.disk_free_bytes_per_mount_point, '/var/lib/lxc')
    )/1024/1024/1024 as min_disk_free_gb, -- Convert to GB for readability
    
    -- Cluster context from join
    c.cluster_name,
    c.worker_node_type,
    c.worker_count
  FROM system.compute.node_timeline nt
  LEFT JOIN system.compute.clusters c 
    ON nt.cluster_id = c.cluster_id 
    AND nt.workspace_id = c.workspace_id
  WHERE 1=1
    -- Time window filter
    AND nt.start_time >= CURRENT_TIMESTAMP - INTERVAL 24 hour
    -- Optional filters (uncomment and modify as needed)
    AND nt.workspace_id = '1444828305810485'
    -- AND nt.cluster_id = '1226-175906-bn5rblc1'
),

aggregated_metrics AS (
  SELECT 
    workspace_id,
    cluster_id,
    instance_id,
    driver,
    cluster_name,
    worker_node_type,
    worker_count,
    
    -- CPU Metrics
    AVG(total_cpu_percent) as avg_cpu_usage,
    MAX(total_cpu_percent) as max_cpu_usage,
    AVG(cpu_wait_percent) as avg_cpu_wait,
    
    -- Memory Metrics
    AVG(mem_used_percent) as avg_memory_usage,
    MAX(mem_used_percent) as max_memory_usage,
    MAX(mem_swap_percent) as max_swap_usage,
    
    -- Network Metrics
    AVG(network_received_mb + network_sent_mb) as avg_network_throughput_mb,
    MAX(network_received_mb + network_sent_mb) as max_network_throughput_mb,
    
    -- Disk Metrics
    MIN(min_disk_free_gb) as min_disk_free_gb,
    
    -- Time range context
    MIN(start_time) as period_start,
    MAX(start_time) as period_end
  FROM base_metrics
  GROUP BY 
    workspace_id,
    cluster_id,
    instance_id,
    driver,
    cluster_name,
    worker_node_type,
    worker_count
)

SELECT 
  -- Identity and Context
  workspace_id,
  cluster_name,
  cluster_id,
  instance_id,
  CASE WHEN driver THEN 'Driver' ELSE 'Worker' END as node_type,
  worker_node_type,
  worker_count,
  
  -- Performance Metrics (rounded for readability)
  ROUND(avg_cpu_usage, 2) as avg_cpu_usage,
  ROUND(max_cpu_usage, 2) as max_cpu_usage,
  ROUND(avg_cpu_wait, 2) as avg_cpu_wait,
  ROUND(avg_memory_usage, 2) as avg_memory_usage,
  ROUND(max_memory_usage, 2) as max_memory_usage,
  ROUND(avg_network_throughput_mb, 2) as avg_network_mb,
  ROUND(min_disk_free_gb, 2) as min_disk_free_gb,
  
  -- Enhanced Status Indicators with Severity Levels
  CASE 
    WHEN avg_cpu_usage > 80 THEN 'CRITICAL: High sustained CPU usage (' || ROUND(avg_cpu_usage, 1) || '%)'
    WHEN max_cpu_usage > 90 THEN 'WARNING: CPU spikes detected (' || ROUND(max_cpu_usage, 1) || '%)'
    WHEN avg_cpu_wait > 20 THEN 'WARNING: High I/O wait times (' || ROUND(avg_cpu_wait, 1) || '%)'
    ELSE 'OK (CPU avg: ' || ROUND(avg_cpu_usage, 1) || '%)'
  END as cpu_status,
  
  CASE 
    WHEN avg_memory_usage > 85 THEN 'CRITICAL: Severe memory pressure (' || ROUND(avg_memory_usage, 1) || '%)'
    WHEN avg_memory_usage > 75 THEN 'WARNING: High memory usage (' || ROUND(avg_memory_usage, 1) || '%)'
    WHEN max_swap_usage > 0 THEN 'WARNING: Memory swapping detected'
    ELSE 'OK (Memory avg: ' || ROUND(avg_memory_usage, 1) || '%)'
  END as memory_status,
  
  CASE 
    WHEN avg_network_throughput_mb > 100 THEN 'WARNING: High network utilization (' || ROUND(avg_network_throughput_mb, 1) || ' MB/s)'
    WHEN max_network_throughput_mb > 500 THEN 'WARNING: Network spikes detected (' || ROUND(max_network_throughput_mb, 1) || ' MB/s)'
    ELSE 'OK (Network avg: ' || ROUND(avg_network_throughput_mb, 1) || ' MB/s)'
  END as network_status,
  
  -- Added Disk Status
  CASE 
    WHEN min_disk_free_gb < 10 THEN 'CRITICAL: Very low disk space (' || ROUND(min_disk_free_gb, 1) || ' GB free)'
    WHEN min_disk_free_gb < 50 THEN 'WARNING: Low disk space (' || ROUND(min_disk_free_gb, 1) || ' GB free)'
    ELSE 'OK (Disk space: ' || ROUND(min_disk_free_gb, 1) || ' GB free)'
  END as disk_status,
  
  -- Time Context
  period_start as monitoring_start,
  period_end as monitoring_end,
  
  -- Overall Health Score (0-100, lower is worse)
  100 - (
    CASE 
      WHEN avg_cpu_usage > 80 THEN 30
      WHEN max_cpu_usage > 90 THEN 15
      WHEN avg_cpu_wait > 20 THEN 10
      ELSE 0
    END +
    CASE 
      WHEN avg_memory_usage > 85 THEN 30
      WHEN avg_memory_usage > 75 THEN 15
      WHEN max_swap_usage > 0 THEN 10
      ELSE 0
    END +
    CASE 
      WHEN avg_network_throughput_mb > 100 THEN 20
      WHEN max_network_throughput_mb > 500 THEN 10
      ELSE 0
    END +
    CASE 
      WHEN min_disk_free_gb < 10 THEN 20
      WHEN min_disk_free_gb < 50 THEN 10
      ELSE 0
    END
  ) as health_score

FROM aggregated_metrics
WHERE period_start >= CURRENT_TIMESTAMP - INTERVAL 24 hour
ORDER BY 
  -- Prioritize issues by severity
  CASE 
    WHEN avg_cpu_usage > 80 OR avg_memory_usage > 85 OR avg_network_throughput_mb > 100 OR min_disk_free_gb < 10 THEN 1
    WHEN max_cpu_usage > 90 OR avg_memory_usage > 75 OR max_network_throughput_mb > 500 OR min_disk_free_gb < 50 THEN 2
    ELSE 3
  END,
  workspace_id,
  cluster_name,
  driver DESC;

In [0]:
-- What clusters currently have autoscaling turned on with the max workers set to a value higher than 10?
with clusters_current as (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY cluster_id
      ORDER BY
        change_time DESC
    ) AS row_num
  FROM
    system.compute.clusters QUALIFY row_num = 1
)
select
  *
from
  clusters_current
where
  max_autoscale_workers >= 10
  and delete_time is null



In [0]:
-- What clusters currently don’t have auto-termination enabled or their auto-termination timeout longer than two hours?
with clusters_current as (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY cluster_id
      ORDER BY
        change_time DESC
    ) AS row_num
  FROM
    system.compute.clusters QUALIFY row_num = 1
)
SELECT
  *
FROM
  clusters_current
where
  (
    auto_termination_minutes is null
    or auto_termination_minutes > 120
  )
  and cluster_name not like 'job-%' -- don't want job clusters
  and delete_time is null